# Fresh take

Trying to get this right.  2024/02/18

## Setup an inventory

In [174]:
import os
import yaml
import copy
import shutil
import numpy as np
import pandas as pd
import importlib
import NewDev as ND
import helperFunctions as hf
importlib.reload(ND)
importlib.reload(hf)
                          
dbpth = os.path.join(os.getcwd(),'test')
shutil.rmtree(dbpth)
os.mkdir(dbpth)

si = ND.siteInventory(dbpth,siteID='SCL',description='Swiss Cheese Lake Eddy Covariance Station',
                latitude='N 69 13.5850',longitude='W 135 15.1144',
                PI = 'June Skeeter, Peter Morse',dateEstablished='2024-07-20')
si = ND.siteInventory(dbpth,siteID='SCL',measurementType='TS',replicateID='SSM',
                  latitude='N 69 13.5239',longitude='W 135 15.1358')
si = ND.siteInventory(dbpth,siteID='SCL',measurementType='TS',replicateID='WSM',
                  latitude='N 69 13.5521',longitude='W 135 15.1350')
si.siteInventory

c:\Users\User\GSC_Work\EC_dataPipeline\test
c:\Users\User\GSC_Work\EC_dataPipeline\test
                siteID                                description  name  \
SCL-.site-.base    SCL  Swiss Cheese Lake Eddy Covariance Station  None   

                                        PI dateEstablished landCoverType  \
SCL-.site-.base  June Skeeter, Peter Morse      2024-07-20          None   

                  longitude   latitude measurementType replicateID  
SCL-.site-.base -135.251907  69.226417           .site       .base  
c:\Users\User\GSC_Work\EC_dataPipeline\test
                siteID                                description  name  \
SCL-.site-.base    SCL  Swiss Cheese Lake Eddy Covariance Station  None   
SCL-TS-SSM         SCL                                       None  None   

                                        PI dateEstablished landCoverType  \
SCL-.site-.base  June Skeeter, Peter Morse      2024-07-20          None   
SCL-TS-SSM                            None      

{'SCL-.site-.base': {'siteID': 'SCL',
  'description': 'Swiss Cheese Lake Eddy Covariance Station',
  'name': None,
  'PI': 'June Skeeter, Peter Morse',
  'dateEstablished': '2024-07-20',
  'landCoverType': None,
  'longitude': -135.2519067,
  'latitude': 69.2264167,
  'measurementType': '.site',
  'replicateID': '.base'},
 'SCL-TS-SSM': {'siteID': 'SCL',
  'description': None,
  'name': None,
  'PI': None,
  'dateEstablished': None,
  'landCoverType': None,
  'longitude': -135.2522633,
  'latitude': 69.2253983,
  'measurementType': 'TS',
  'replicateID': 'SSM'},
 'SCL-TS-WSM': {'siteID': 'SCL',
  'description': None,
  'name': None,
  'PI': None,
  'dateEstablished': None,
  'landCoverType': None,
  'longitude': -135.25225,
  'latitude': 69.2258683,
  'measurementType': 'TS',
  'replicateID': 'WSM'}}

In [ ]:
import os
import re
import yaml
import copy
import shutil
import datetime
import importlib
from typing import Literal
from dataclasses import dataclass, field
import helperFunctions as hf
importlib.reload(hf)

@dataclass(kw_only=True)
class record:
    fillChar = '_'
    sepChar = '-'
    siteInventory = {}

    siteID: str
    measurementType: str = fillChar
    replicateID: str = fillChar
    description: str = field(default=None,repr=False)
    dateEst: str = field(default=None,repr=False)
    name: str = field(default=None,repr=False)
    PI: str = field(default=None,repr=False)
    dateEstablished: str = field(default=None,repr=False)
    landCoverType: str = field(default=None,repr=False)
    longitude: float = field(default=None,repr=False)
    latitude: float = field(default=None,repr=False)
    def __post_init__(self):
        self.MRO_fields = {v.__name__:list(v.__dict__['__dataclass_fields__'].keys()) 
                      for v in type(self).__mro__
                      if v.__name__ != 'object'}
        for k in self.__dataclass_fields__.keys():
            if self.__dataclass_fields__[k].repr and self.__dict__[k] is not None:
                self.__dict__[k] = re.sub('[^0-9a-zA-Z]+',self.fillChar, self.__dict__[k])
        self.measurementID = self.sepChar.join([str(self.__dict__[k]) for k in self.__dataclass_fields__.keys() 
                                       if self.__dataclass_fields__[k].repr and self.__dict__[k] is not None
                                       ])
        if self.measurementID not in self.siteInventory.keys() or self.overwrite:
            self.siteInventory[self.measurementID] = {}
            for f,v in self.__dataclass_fields__.items():
                if (self.__dict__[f] != self.fillChar and (v.kw_only and self.__dict__[f] is not None or  
                    self.measurementID.endswith(self.sepChar.join([self.fillChar,self.fillChar]))) and
                    f in self.MRO_fields['record']):
                    self.siteInventory[self.measurementID][f] = self.__dict__[f]
        
@dataclass
class siteInventory(record):
    siteInventory = {}
    source: str = field(repr=False)
    overwrite: bool = field(default=False,repr=False)

    def __post_init__(self):
        self.load(self.source)
        super().__post_init__()
        self.save(self.source)

    def load(self,source):
        if os.path.isfile(os.path.join(source,'siteInventory.yml')):
            with open(os.path.join(source,'siteInventory.yml')) as f:
                inv = yaml.safe_load(f)
                if inv is not None:
                    self.siteInventory = hf.unpackDict(inv,'-',limit=2)
        elif os.path.isdir(source):
            with open(os.path.join(source,'siteInventory.yml'),'w+') as f:
                yaml.safe_dump(self.siteInventory,f)
        elif os.path.isfile(source):
            print('csv?')
            
    def save(self,source):
        with open(os.path.join(source,'siteInventory.yml'),'w+') as f:
            # Sort alphabetically by ID, maintaining order of metadata
            # Hacked up to maintain desirable sort order regardless of case or pattern used to represent "blank" values
            self.siteInventory = {key.replace(self.fillChar,' '):value for key,value in self.siteInventory.items()}
            self.siteInventory = {key.replace(' ',self.fillChar):value for key,value in dict(sorted(self.siteInventory.items())).items()}
            self.siteInventory = hf.packDict(copy.deepcopy(self.siteInventory),'-',limit=2,order=1)
            yaml.safe_dump(self.siteInventory,f,sort_keys=False)


dbpth = os.path.join(os.getcwd(),'test')
shutil.rmtree()
M = siteInventory(source=dbpth,siteID='SCL',description='Swiss Cheese Lake Eddy Covariance Station',
                  latitude='N 69 13.5850',longitude='W 135 15.1144',
                #   siteVisitDates=['2024-07-24','2024-09-12','2024-09-14']
                PI = 'June Skeeter, Peter Morse',
                dateEstablished='2024-07-20'
                  )
M = siteInventory(source=dbpth,siteID='SCL',measurementType='TS',replicateID='SSM',
                  latitude='N 69 13.5239',longitude='W 135 15.1350')
M = siteInventory(source=dbpth,siteID='SCL',measurementType='TS',replicateID='WSM',
                  latitude='N 69 13.5521',longitude='W 135 15.1350')
# M.measurementID
M.siteInventory

{'SCL': {'_': {'_': {'siteID': 'SCL',
    'description': 'Swiss Cheese Lake Eddy Covariance Station',
    'dateEst': None,
    'name': None,
    'PI': None,
    'dateEstablished': None,
    'landCoverType': None,
    'longitude': 'W 135 15.1144',
    'latitude': 'N 69 13.5850'}},
  'TS': {'SSM': {'siteID': 'SCL',
    'measurementType': 'TS',
    'replicateID': 'SSM',
    'longitude': 'W 135 15.1350',
    'latitude': 'N 69 13.5239'},
   'WSM': {'siteID': 'SCL',
    'measurementType': 'TS',
    'replicateID': 'WSM',
    'longitude': 'W 135 15.1350',
    'latitude': 'N 69 13.5521'}}}}

In [16]:
hf.unpackDict(M.siteInventory,'-',limit=2)

{'SCL-_-_': {'siteID': 'SCL',
  'description': 'Swiss Cheese Lake Eddy Covariance Station',
  'dateEst': None,
  'name': None,
  'PI': None,
  'dateEstablished': None,
  'landCoverType': None,
  'longitude': 'W 135 15.1144',
  'latitude': 'N 69 13.5850'},
 'SCL-TS-SSM': {'siteID': 'SCL',
  'measurementType': 'TS',
  'replicateID': 'SSM',
  'longitude': 'W 135 15.1350',
  'latitude': 'N 69 13.5239'},
 'SCL-TS-WSM': {'siteID': 'SCL',
  'measurementType': 'TS',
  'replicateID': 'WSM',
  'longitude': 'W 135 15.1350',
  'latitude': 'N 69 13.5521'}}

In [ ]:

@dataclass
class filling:
    r: str = '1'
    t: str = 'b'

    def __post_init__(self):
        j  = '1'
        print('!!!',type(self).__base__.__name__)



@dataclass
class pie(filling):
    a: str = '1'
    b: str = 'b'

    def __post_init__(self):
        print('!!',type(self).)#.__base__.__name__)
        super().__post_init__()


@dataclass
class food(pie):

    def __post_init__(self):
        print('!',type(self).__base__.__name__)
        super().__post_init__()
        self.c = 1
        MRO_fields = {v.__name__:list(v.__dict__['__dataclass_fields__'].keys()) 
                      for v in type(self).__mro__
                      if v.__name__ != 'object'}
        print(MRO_fields)
        # print(self.__doc__)
        # print(list(type(self).__mro__[0].__dict__['__dataclass_fields__'].keys()))
    # def __init__(self,**kwargs):
    #     super().__init__()
    #     # pass

food()
# kwargs

! pie
!! food
!!! pie
{'food': ['r', 't', 'a', 'b'], 'pie': ['r', 't', 'a', 'b'], 'filling': ['r', 't']}


food(r='1', t='b', a='1', b='b')

In [31]:
@dataclass(kw_only=True)
class tt:
    a: str = 'pi'

    def __post_init__(self):
        print(self.__dataclass_fields__)

@dataclass
class tst(tt):
    source: str = field(repr=False)
    def __post_init__(self):
        return super().__post_init__()
    
tst(source='C:/a')

{'a': Field(name='a',type=<class 'str'>,default='pi',default_factory=<dataclasses._MISSING_TYPE object at 0x000001992830AE40>,init=True,repr=True,hash=None,compare=True,metadata=mappingproxy({}),kw_only=True,_field_type=_FIELD), 'source': Field(name='source',type=<class 'str'>,default=<dataclasses._MISSING_TYPE object at 0x000001992830AE40>,default_factory=<dataclasses._MISSING_TYPE object at 0x000001992830AE40>,init=True,repr=False,hash=None,compare=True,metadata=mappingproxy({}),kw_only=False,_field_type=_FIELD)}


tst(a='pi')

## File Management

In [ ]:

source = r'..\SCL_2024'
# print(source)
for dir,_,files in os.walk(source):
    # print(dir)
    for f in files:
        if f.endswith('.csv'):
            file = os.path.join(dir,f)
            for key in ['WSM']:
                if key in file:
                    fn = file

# Database

In [51]:
fn

import rawDataFile as rawDataFile
importlib.reload(rawDataFile)

rd = rawDataFile.hoboCSV(source=fn,excludePattern={r"LGR(.*?)LBL: ":'',
                                                   r'\)':'cm'})
rd.Metadata

Standardizing and documenting traces
Re-named:  #  to:  _
observation(ignore=True, name_in='#', unit_in=None, safe_name='_', dtype='<i8', variableDescription=None)
Re-named:  Date Time, GMT+00:00  to:  Date_Time_GMT_00_00
observation(ignore=True, name_in='Date Time, GMT+00:00', unit_in=None, safe_name='Date_Time_GMT_00_00', dtype='|O', variableDescription=None)
Re-named:  Temp, °C (5cm  to:  Temp_C_5cm
observation(ignore=False, name_in='Temp, °C (5cm', unit_in=None, safe_name='Temp_C_5cm', dtype='<f8', variableDescription=None)
Re-named:  Temp, °C (25cm  to:  Temp_C_25cm
observation(ignore=False, name_in='Temp, °C (25cm', unit_in=None, safe_name='Temp_C_25cm', dtype='<f8', variableDescription=None)
Re-named:  Temp, °C (50cm  to:  Temp_C_50cm
observation(ignore=False, name_in='Temp, °C (50cm', unit_in=None, safe_name='Temp_C_50cm', dtype='<f8', variableDescription=None)
Re-named:  Temp, °C (100cm  to:  Temp_C_100cm
observation(ignore=False, name_in='Temp, °C (100cm', unit_in=None, safe_

{'verbose': True,
 'loggerName': None,
 'frequency': '30min',
 'timeZone': 'UTC',
 'yearfirst': True,
 'Variables': {'_': {'ignore': True,
   'name_in': '#',
   'unit_in': None,
   'safe_name': '_',
   'dtype': '<i8',
   'variableDescription': None},
  'Date_Time_GMT_00_00': {'ignore': True,
   'name_in': 'Date Time, GMT+00:00',
   'unit_in': None,
   'safe_name': 'Date_Time_GMT_00_00',
   'dtype': '|O',
   'variableDescription': None},
  'Temp_C_5cm': {'ignore': False,
   'name_in': 'Temp, °C (5cm',
   'unit_in': None,
   'safe_name': 'Temp_C_5cm',
   'dtype': '<f8',
   'variableDescription': None},
  'Temp_C_25cm': {'ignore': False,
   'name_in': 'Temp, °C (25cm',
   'unit_in': None,
   'safe_name': 'Temp_C_25cm',
   'dtype': '<f8',
   'variableDescription': None},
  'Temp_C_50cm': {'ignore': False,
   'name_in': 'Temp, °C (50cm',
   'unit_in': None,
   'safe_name': 'Temp_C_50cm',
   'dtype': '<f8',
   'variableDescription': None},
  'Temp_C_100cm': {'ignore': False,
   'name_in': 'T

In [63]:
import shutil
import database as pyDB
importlib.reload(pyDB)

dbpth = os.path.join(os.getcwd(),'test')
if os.path.isdir(dbpth):
    shutil.rmtree(dbpth)

# db = pyDB.database(projectPath=dbpth,
#     # siteID=['SCL'],
#     # Years=[str(y) for y in range(2024,2025)],
#     # verbose=False
#     )
# db.
n = pyDB.rawDatabaseImport(projectPath=dbpth,measurementID=M.measurementID,dataIn=rd.Data,metadataIn=rd.Metadata)
n.dataIn

Initializing empty database
Creating folder:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\_metadataFile.yml
Creating file:  c:\Users\jskeeter\gsc-permafrost\EC_dataPipeline\test\_logFile.txt
_ {'ignore': True, 'name_in': '#', 'unit_in': None, 'safe_name': '_', 'dtype': '<i8', 'variableDescription': None}
Date_Time_GMT_00_00 {'ignore': True, 'name_in': 'Date Time, GMT+00:00', 'unit_in': None, 'safe_name': 'Date_Time_GMT_00_00', 'dtype': '|O', 'variableDescription': None}
Temp_C_5cm {'ignore': False, 'name_in': 'Temp, °C (5cm', 'unit_in': None, 'safe_name': 'Temp_C_5cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_25cm {'ignore': False, 'name_in': 'Temp, °C (25cm', 'unit_in': None, 'safe_name': 'Temp_C_25cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_50cm {'ignore': False, 'name_in': 'Temp, °C (50cm', 'unit_in': None, 'safe_name': 'Temp_C_50cm', 'dtype': '<f8', 'variableDescription': None}
Temp_C_100cm {'ignore': False, 'name_in': 'Temp, °C (100cm', 'unit_in':

,Temp_C_5cm,Temp_C_25cm,Temp_C_50cm,Temp_C_100cm
2024-07-20 21:00:00,7.217,2.423,-0.032,-1.470
2024-07-20 21:30:00,11.759,6.712,-0.060,-1.470
2024-07-20 22:00:00,11.467,5.257,-0.060,-1.470
2024-07-20 22:30:00,11.029,4.428,-0.088,-1.470
2024-07-20 23:00:00,10.907,4.011,-0.088,-1.470
...,...,...,...,...
2024-09-13 16:00:00,4.869,3.670,0.412,-0.845
2024-09-13 16:30:00,4.869,3.643,0.412,-0.845
2024-09-13 17:00:00,4.869,3.643,0.412,-0.845
2024-09-13 17:30:00,4.895,3.617,0.412,-0.845


In [61]:
siteID,subsiteID='a'.split('-',1)

ValueError: not enough values to unpack (expected 2, got 1)